# Import datasets taken from MOH

In [1]:
import pandas as pd

In [2]:
cases_df = pd.read_csv("cases_state.csv")
cases_df.head()

,date,state,cases_new,cases_import,cases_recovered,cases_active,cases_cluster,cases_pvax,cases_fvax,cases_child,cases_adolescent,cases_adult,cases_elderly
0,2020-01-25,Johor,4,4,0,4,0,0,0,0,0,1,0
1,2020-01-25,Kedah,0,0,0,0,0,0,0,0,0,0,0
2,2020-01-25,Kelantan,0,0,0,0,0,0,0,0,0,0,0
3,2020-01-25,Melaka,0,0,0,0,0,0,0,0,0,0,0
4,2020-01-25,Negeri Sembilan,0,0,0,0,0,0,0,0,0,0,0


In [3]:
deaths_df = pd.read_csv("deaths_state.csv")
deaths_df.head()

,date,state,deaths_new,deaths_bid,deaths_new_dod,deaths_bid_dod,deaths_pvax,deaths_fvax,deaths_tat
0,2020-03-17,Johor,1,0,1,0,0,0,0
1,2020-03-17,Kedah,0,0,0,0,0,0,0
2,2020-03-17,Kelantan,0,0,0,0,0,0,0
3,2020-03-17,Melaka,0,0,0,0,0,0,0
4,2020-03-17,Negeri Sembilan,0,0,0,0,0,0,0


In [4]:
beds_df = pd.read_csv("hospital.csv")
beds_df.head()

,date,state,beds,beds_covid,beds_noncrit,admitted_pui,admitted_covid,admitted_total,discharged_pui,discharged_covid,discharged_total,hosp_covid,hosp_pui,hosp_noncovid
0,2020-03-24,Johor,1440,434,1408,0,0,0,0,0,0,18,9,1
1,2020-03-24,Kedah,1218,143,1183,0,0,0,0,0,0,46,5,12
2,2020-03-24,Kelantan,1463,280,1424,9,6,15,1,0,1,78,58,23
3,2020-03-24,Melaka,1091,82,1065,0,0,0,0,0,0,19,10,12
4,2020-03-24,Negeri Sembilan,1223,253,1205,0,0,0,0,0,0,90,13,8


In [5]:
icu_df = pd.read_csv("icu.csv")
icu_df.head()

,date,state,beds_icu,beds_icu_rep,beds_icu_total,beds_icu_covid,vent,vent_port,icu_covid,icu_pui,icu_noncovid,vent_covid,vent_pui,vent_noncovid,vent_used,vent_port_used
0,2020-03-24,Johor,10,22,32,10,11,2,0,0,1,0,0,0,8,0
1,2020-03-24,Kedah,35,0,35,16,34,0,2,1,12,1,1,0,19,0
2,2020-03-24,Kelantan,39,0,39,20,42,4,1,0,20,1,0,0,26,0
3,2020-03-24,Melaka,26,0,26,12,23,4,0,2,12,0,1,0,13,5
4,2020-03-24,Negeri Sembilan,18,0,18,10,14,9,0,0,8,0,0,0,1,6


Drop extra columns and convert date to datetime

In [6]:
cases_df = cases_df.loc[:, cases_df.columns.intersection(['date','state', 'cases_new'])]
cases_df['date'] = pd.to_datetime(cases_df['date'])
cases_df.head()

,date,state,cases_new
0,2020-01-25,Johor,4
1,2020-01-25,Kedah,0
2,2020-01-25,Kelantan,0
3,2020-01-25,Melaka,0
4,2020-01-25,Negeri Sembilan,0


In [7]:
deaths_df = deaths_df.loc[:, deaths_df.columns.intersection(['date','state', 'deaths_new'])]
deaths_df['date'] = pd.to_datetime(deaths_df['date'])
deaths_df.head()

,date,state,deaths_new
0,2020-03-17,Johor,1
1,2020-03-17,Kedah,0
2,2020-03-17,Kelantan,0
3,2020-03-17,Melaka,0
4,2020-03-17,Negeri Sembilan,0


In [8]:
beds_df = pd.merge(beds_df, icu_df, on=['date','state'])
beds_df = beds_df.loc[:, beds_df.columns.intersection(['date','state', 'beds', 'beds_covid', 'beds_icu_covid'])]
beds_df['date'] = pd.to_datetime(beds_df['date'])
beds_df.head()

,date,state,beds,beds_covid,beds_icu_covid
0,2020-03-24,Johor,1440,434,10
1,2020-03-24,Kedah,1218,143,16
2,2020-03-24,Kelantan,1463,280,20
3,2020-03-24,Melaka,1091,82,12
4,2020-03-24,Negeri Sembilan,1223,253,10


# Combine datasets

In [9]:
from datetime import date, timedelta, datetime
# format: y/m/d
sdate = date(2020,1,1)
edate = date(2021,10,14)

states = list(cases_df['state'].unique())

new_df=pd.DataFrame(columns=['date','state', 'cases_new', 'deaths_new', 'beds', 'beds_covid', 'beds_icu_covid'])
row_count = 0
for date in pd.date_range(sdate,edate-timedelta(days=1),freq='d'):
    for state in states:
        row_count += 1
        new_df.loc[row_count, 'date'] = datetime.strftime(date, '%Y-%m-%d')
        new_df.loc[row_count, 'state'] = state
# new_df['date'] = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
new_df.head(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
1,2020-01-01,Johor,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,Kedah,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,Kelantan,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,Melaka,NaN,NaN,NaN,NaN,NaN
5,2020-01-01,Negeri Sembilan,NaN,NaN,NaN,NaN,NaN
6,2020-01-01,Pahang,NaN,NaN,NaN,NaN,NaN
7,2020-01-01,Perak,NaN,NaN,NaN,NaN,NaN
8,2020-01-01,Perlis,NaN,NaN,NaN,NaN,NaN
9,2020-01-01,Pulau Pinang,NaN,NaN,NaN,NaN,NaN
10,2020-01-01,Sabah,NaN,NaN,NaN,NaN,NaN


Take data from old datasets into new datasets.

In [10]:
import datetime
def transpose(new_df, old_df, cols):
    for index, row in new_df.iterrows():
        current_date = new_df.loc[index, 'date']
        current_state = new_df.loc[index, 'state']
        for index1, row1 in old_df.loc[old_df['date'] == current_date].iterrows():
            state = old_df.loc[index1, 'state']
            if current_state == state:
                for col in cols:
                    new_df.loc[index, col] = old_df.loc[index1, col]

transpose(new_df, cases_df, ['cases_new'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
10413,2021-10-12,Terengganu,420,NaN,NaN,NaN,NaN
10414,2021-10-12,W.P. Kuala Lumpur,253,NaN,NaN,NaN,NaN
10415,2021-10-12,W.P. Labuan,2,NaN,NaN,NaN,NaN
10416,2021-10-12,W.P. Putrajaya,32,NaN,NaN,NaN,NaN
10417,2021-10-13,Johor,703,NaN,NaN,NaN,NaN
10418,2021-10-13,Kedah,489,NaN,NaN,NaN,NaN
10419,2021-10-13,Kelantan,958,NaN,NaN,NaN,NaN
10420,2021-10-13,Melaka,188,NaN,NaN,NaN,NaN
10421,2021-10-13,Negeri Sembilan,167,NaN,NaN,NaN,NaN
10422,2021-10-13,Pahang,342,NaN,NaN,NaN,NaN


In [11]:
transpose(new_df, deaths_df, ['deaths_new'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
10413,2021-10-12,Terengganu,420,0,NaN,NaN,NaN
10414,2021-10-12,W.P. Kuala Lumpur,253,1,NaN,NaN,NaN
10415,2021-10-12,W.P. Labuan,2,0,NaN,NaN,NaN
10416,2021-10-12,W.P. Putrajaya,32,0,NaN,NaN,NaN
10417,2021-10-13,Johor,703,9,NaN,NaN,NaN
10418,2021-10-13,Kedah,489,2,NaN,NaN,NaN
10419,2021-10-13,Kelantan,958,6,NaN,NaN,NaN
10420,2021-10-13,Melaka,188,3,NaN,NaN,NaN
10421,2021-10-13,Negeri Sembilan,167,2,NaN,NaN,NaN
10422,2021-10-13,Pahang,342,3,NaN,NaN,NaN


In [12]:
transpose(new_df, beds_df, ['beds', 'beds_covid', 'beds_icu_covid'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid
10413,2021-10-12,Terengganu,420,0,1557,491,38
10414,2021-10-12,W.P. Kuala Lumpur,253,1,3963,675,43
10415,2021-10-12,W.P. Labuan,2,0,122,106,18
10416,2021-10-12,W.P. Putrajaya,32,0,609,84,6
10417,2021-10-13,Johor,703,9,5160,2753,102
10418,2021-10-13,Kedah,489,2,2664,1259,130
10419,2021-10-13,Kelantan,958,6,2815,923,85
10420,2021-10-13,Melaka,188,3,1317,651,38
10421,2021-10-13,Negeri Sembilan,167,2,1749,1084,88
10422,2021-10-13,Pahang,342,3,2066,709,81


In [13]:
new_df.dtypes

date              object
state             object
cases_new         object
deaths_new        object
beds              object
beds_covid        object
beds_icu_covid    object
dtype: object

In [14]:
new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]] = new_df[["cases_new", "deaths_new", "beds", "beds_covid", "beds_icu_covid"]].apply(pd.to_numeric)
new_df.dtypes

date               object
state              object
cases_new         float64
deaths_new        float64
beds              float64
beds_covid        float64
beds_icu_covid    float64
dtype: object

In [15]:
new_df=new_df.sort_values(['date']).reset_index(drop=True)
new_df["cum_cases"]=new_df.groupby(['state'])['cases_new'].cumsum(axis=0)
new_df["cum_deaths"]=new_df.groupby(['state'])['deaths_new'].cumsum(axis=0)
new_df.sort_values(['date', 'state'])
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid,cum_cases,cum_deaths
10412,2021-10-12,Kelantan,1028.0,11.0,2815.0,919.0,85.0,127408.0,918.0
10413,2021-10-12,Kedah,517.0,6.0,2664.0,1259.0,120.0,141342.0,1912.0
10414,2021-10-12,Johor,665.0,15.0,5160.0,2753.0,102.0,206811.0,3455.0
10415,2021-10-12,Perlis,57.0,1.0,487.0,241.0,11.0,4716.0,91.0
10416,2021-10-13,W.P. Kuala Lumpur,309.0,0.0,3963.0,675.0,45.0,189085.0,2522.0
10417,2021-10-13,Terengganu,499.0,4.0,1557.0,491.0,37.0,63523.0,412.0
10418,2021-10-13,Selangor,1449.0,3.0,5902.0,1945.0,179.0,693754.0,9497.0
10419,2021-10-13,Sarawak,1165.0,16.0,4055.0,1406.0,151.0,227675.0,1041.0
10420,2021-10-13,Sabah,751.0,6.0,4109.0,1797.0,227.0,207506.0,2369.0
10421,2021-10-13,Pulau Pinang,474.0,3.0,1985.0,710.0,93.0,138200.0,1497.0


In [16]:
new_df['state'] = new_df['state'].replace(['Pulau Pinang'],'Penang')
new_df['state'] = new_df['state'].replace(['W.P. Kuala Lumpur'],'Kuala Lumpur')
new_df['state'] = new_df['state'].replace(['W.P. Labuan'],'Labuan')
new_df['state'] = new_df['state'].replace(['W.P. Putrajaya'],'Putrajaya')
new_df['state'] = new_df['state'].replace(['Melaka'],'Malacca')
new_df.tail(20)

,date,state,cases_new,deaths_new,beds,beds_covid,beds_icu_covid,cum_cases,cum_deaths
10412,2021-10-12,Kelantan,1028.0,11.0,2815.0,919.0,85.0,127408.0,918.0
10413,2021-10-12,Kedah,517.0,6.0,2664.0,1259.0,120.0,141342.0,1912.0
10414,2021-10-12,Johor,665.0,15.0,5160.0,2753.0,102.0,206811.0,3455.0
10415,2021-10-12,Perlis,57.0,1.0,487.0,241.0,11.0,4716.0,91.0
10416,2021-10-13,Kuala Lumpur,309.0,0.0,3963.0,675.0,45.0,189085.0,2522.0
10417,2021-10-13,Terengganu,499.0,4.0,1557.0,491.0,37.0,63523.0,412.0
10418,2021-10-13,Selangor,1449.0,3.0,5902.0,1945.0,179.0,693754.0,9497.0
10419,2021-10-13,Sarawak,1165.0,16.0,4055.0,1406.0,151.0,227675.0,1041.0
10420,2021-10-13,Sabah,751.0,6.0,4109.0,1797.0,227.0,207506.0,2369.0
10421,2021-10-13,Penang,474.0,3.0,1985.0,710.0,93.0,138200.0,1497.0


In [17]:
new_df.to_csv("daily.csv", index=False)